In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [4]:
df = pd.read_csv('datasets/SatisfactionClients/cdiscount2.csv')
df_test = pd.read_csv('datasets/SatisfactionClients/amazon_test1.csv')
df.sample(3)

,note,commentaire
4331,5,satisfait
4109,2,après plusieurs jours tentative l'achat trois ...
42155,5,jusqu' présent très satisfait services cdiscou...


In [5]:
df.isna().sum()

note            0
commentaire    54
dtype: int64

In [6]:
df[df['commentaire'].isna()==True]

,note,commentaire
156,5,NaN
364,4,NaN
473,5,NaN
476,5,NaN
1459,5,NaN
2040,3,NaN
2144,5,NaN
2293,5,NaN
2749,5,NaN
2920,4,NaN


In [7]:
df = df[df['commentaire'].isna()==False]

In [12]:
# séparation de la variable cible et des variables explicatives
X = df['commentaire']
y = df['note']

In [13]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [14]:
# conversion des chaines de caratères en tokens numériques
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=5000, ngram_range=[2, 3])
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

In [15]:
np.save('datasets/SatisfactionClients/vector_matrix_ngram_2_3', X_train)

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=50)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(clf.score(X_test, y_test), end='\n\n')

cr = classification_report(y_test, y_pred)
print(cr)

cm = pd.crosstab(y_test, y_pred, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test)

for i in cm_test.index:
    for j in cm_test.columns :
        if (i==j and cm_test.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6347702510658456

              precision    recall  f1-score   support

           1       0.66      0.63      0.64      1956
           2       0.08      0.02      0.04       509
           3       0.22      0.15      0.17      1102
           4       0.34      0.19      0.24      3508
           5       0.71      0.88      0.79      9813

    accuracy                           0.63     16888
   macro avg       0.40      0.37      0.38     16888
weighted avg       0.58      0.63      0.59     16888



predictions,1,2,3,4,5
données réelles,,,,,
1,1234,53,134,153,382
2,157,12,87,79,174
3,196,24,160,248,474
4,142,30,185,658,2493
5,153,28,167,809,8656


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,1,2,3,4,5
données réelles,,,,,
1,11,0,3,0,6
2,8,0,6,0,6
3,7,0,1,8,4
4,2,1,0,3,14
5,3,0,0,2,15


predictions,1,2,3,4,5
données réelles,,,,,
1,0.55,0.00,0.15,0.00,0.30
2,0.40,0.00,0.30,0.00,0.30
3,0.35,0.00,0.05,0.40,0.20
4,0.10,0.05,0.00,0.15,0.70
5,0.15,0.00,0.00,0.10,0.75


les prédictions correctes sont supérieures à 50% pour 1 étoile(s) avec 55.00000000000001 %
les prédictions correctes sont supérieures à 50% pour 5 étoile(s) avec 75.0 %


In [29]:
from joblib import dump
dump(clf, 'datasets/SatisfactionClients/model_rf_ngrams_2_3.joblib')

['datasets/SatisfactionClients/model_rf_ngrams_2_3.joblib']

### Evaluation sur 2 sentiments : négatif et positif

* Le DataSet cdiscount a été retraité en renommant, les notes 1 et 2 en 0, les notes 4 et 5 en 1 et en supprimant les notes 3
* Le but étant d'isoler de manière moins complexe les sentiments négatifs et les sentiments positifs et ainsi améliorer la qualité du résultat prédit

In [17]:
df_0_1 = pd.read_csv('datasets/SatisfactionClients/cdiscount_0_1.csv')

In [18]:
df_0_1.isna().sum()

note            0
commentaire    46
dtype: int64

In [19]:
df_0_1 = df_0_1[df_0_1['commentaire'].isna()==False]

In [24]:
# séparation de la variable cible et des variables explicatives
X1 = df_0_1['commentaire']
y1 = df_0_1['note']

In [25]:
# séparation du jeu de données en un dataset d'entrainement et un dataset de test
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, shuffle=True)

In [26]:
X_train1 = vectorizer.transform(X_train1).todense()
X_test1 = vectorizer.transform(X_test1).todense()

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf_0_1 = RandomForestClassifier(n_estimators=50)

clf_0_1.fit(X_train1, y_train1)

y_pred_0_1 = clf_0_1.predict(X_test1)

print(clf_0_1.score(X_test1, y_test1), end='\n\n')

cr = classification_report(y_test1, y_pred_0_1)
print(cr)

cm = pd.crosstab(y_test1, y_pred_0_1, rownames=['données réelles'], colnames=['predictions'])
display(cm)

# vérification des résultats sur un jeu de test externe (100 commentaires amazon également répartis entre les étoiles)
df_test_token = vectorizer.transform(df_test['commentaire']).todense()
y_predict_test = clf_0_1.predict(df_test_token)

cm_test = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'])
display(cm_test)

cm_test2 = pd.crosstab(df_test['note'], y_predict_test, rownames=['données réelles'], colnames=['predictions'], normalize=0)
display(cm_test2)

for i in cm_test2.index:
    for j in cm_test2.columns :
        if (i==j and cm_test2.loc[i, j] > 0.5) :
            print("les prédictions correctes sont supérieures à 50% pour {} étoile(s) avec {} %".format(i, cm_test2.loc[i, j]*100))

C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.9159637216972157

              precision    recall  f1-score   support

           0       0.77      0.68      0.72      2525
           1       0.94      0.96      0.95     13242

    accuracy                           0.92     15767
   macro avg       0.85      0.82      0.84     15767
weighted avg       0.91      0.92      0.91     15767



predictions,0,1
données réelles,,
0,1718,807
1,518,12724


C:\Users\t.fourtouill\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


predictions,0,1
données réelles,,
1,15,5
2,10,10
3,8,12
4,5,15
5,3,17


predictions,0,1
données réelles,,
1,0.75,0.25
2,0.50,0.50
3,0.40,0.60
4,0.25,0.75
5,0.15,0.85


In [28]:
from joblib import dump
dump(clf_0_1, 'datasets/SatisfactionClients/model_rf_0_1_ngrams_2_3.joblib')

['datasets/SatisfactionClients/model_rf_0_1_tfidf_ngrams_2_3.joblib']